# Lab 2: Working with Anthropic API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

#### Step 1: Import libraries and load the environment variables

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [ ]:
client = anthropic.Client()
message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system="You are a helpful assistant.",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=100
)
print(message.content[0].text)

Message(id='msg_019ZvczP4cM5J7RggbDy2ZCo', content=[TextBlock(citations=None, text='The capital of France is Paris. It is located in the north-central part of the country and is the largest city in France. Paris is known for its iconic landmarks like the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and is considered a global center for art, culture, fashion, and cuisine.', type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=20, output_tokens=72, server_tool_use=None, service_tier='standard'))


#### Step 3: Customize the Prompt

Customer service representative

In [49]:
complaint = "You have over charged me for this product. I demand a refund."

In [50]:
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [66]:
system_message = "You are a customer service assistant who help handle cusomer requests."

prompt = f"""
Using the company policy below, draft a polite response to the customer complaint.
- Customer request: "{complaint} "
- Customer policy: "{company_policy}"

Your response should address the customer's concern while adhering to company policy.
Keep your tone friendly and professional, but also firm about the policy.
Adds conditional language ("if eligible")
Still promises quick resolution without overcommitting
Sets expectation that damaged items are handled promptly
Complies with company policy while prioritizing customer satisfaction

Keep your response short and to the point, no more than 100 words.

"""

chat_message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system=system_message,
    messages=[
        {"role": "user", "content": prompt},
        ],
    max_tokens=200
)
print(chat_message.content[0].text)

Here's a draft response:

Dear Valued Customer,

I'm sorry to hear your ceramic mug arrived damaged. We take product quality seriously and want to make this right. Please send us a photo of the broken item, and our team will process a replacement or refund if you are within our 14-day return window. 

Could you provide me with your order number and purchase date? This will help me verify your eligibility and expedite a resolution. We aim to address shipping damage quickly and ensure you receive a perfect product.

Thank you for bringing this to our attention.

Best regards,
Customer Service Team


#### Step 4: Rate the response using a reasoning model

In [68]:
system_message = "You are a senior customer service agent with advanced reasoning capabilities. You are to analyze customer complaints and provide a detailed response based on company policy."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint and company policy
- Professional tone and empathy

"""

reasoning_message = client.messages.create(
    model="claude-sonnet-4-5-20250929", #"claude-3-7-sonnet-latest" deprecated
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    temperature=1,
    system=system_message,
    messages=[
        { "role": "user", "content": reasoning_prompt}
    ]  
)

reasoning_message = reasoning_message.content
print(reasoning_message)


[ThinkingBlock(signature='EpgSCkYIChgCKkAbXl2kf1U63S8qU/oSfui9ujZJIZ1dIykAMlNE0DmlJ0Mh3duLWGqZuNebdQSy7QIJkejYXrIAHcoXM7sswYTlEgxDJTv7eSmFlvkf2b0aDEeszPlvInzOevDQcSIweqCbk8cQOMYO6FDcogjXl/bZkCAlNQ0FF8/kqscvs79gEco7FnQpAVAfAqurVlcmKv8Qn53hlxHyPYJHKiCgGDhc5lSIKi4E2RvRk4EH6nHQd1BqYuhNhDgapWDyh7dpWwTIt94i6CuYB1twXONay994SyTbfyip46a8HcpszI1Dlb4U5GynCZ6ruoBsVMCcqD1CvSaJbnP6UQ7ECRi1aLse7W2jh4DPwUOfOa5v8KAEDoPbqKrMg0wYjO/ipqVYkLWNIjsBsdwApjz82fF22uCfeYw8Sow5Ez1G8x1u5NIKyXYfzjhC3F28LKysYSRQRVwrUIzhZYBzFTJkGUuxIGs64vIUJBP2P2WMkGLLCEgvQ2mWGceZrYSXgpPJbA5RrURYfT6crHGYtRp0tVQ2gemg1zWIz0lyoqs5y1nhWdF6r5SNkaott9XnQue/C05GuKWFS9dSL4xyolSvzXnAiTrBAo3vGdQ2BTKzFl8WEeM8JEepdJJIaT6sb5W+MYdF+s1CLXXBygDfmZ+xNGVf0tb7beNNNB2+73IePQngreIhMMG+D+gEVFoQSqOC1furexS+TpS2YjzUDEyVPc7BcV2MeYCO3jOb709SzGamn6pCyk5rmKVKnWJD74bysmTIct96WLJYnyE341xB2j5sQG7tCwZC/XjwP9A2NW1VOE4FWUeei3iW45QfxRTSl0nQ0S/OdM+6HlPocUB3FtE+TCZHjjRdgQRxN2rSreh1WyT1vlD6u3+YD3EXzyVpmLaQebDGIRdMe92X7rD2giOtNEh/yZoCGo40Xibq8kDP4izVEY2Ws28XdAVkLGA7T10fB2

In [69]:
reasoning_message = reasoning_message[1].text
display(Markdown(reasoning_message))

# Customer Service Response Analysis

## Overall Assessment: **ADEQUATE with Room for Improvement**

---

## Detailed Evaluation

### ✅ **Strengths**

1. **Policy Compliance**
   - Correctly references the 14-day return window
   - Requests order verification (order number, purchase date)
   - Offers both replacement and refund options as appropriate

2. **Professional Structure**
   - Proper greeting and closing
   - Clear next steps requested from customer
   - Acknowledges the problem

### ⚠️ **Areas of Concern**

1. **Prioritization Issues**
   - For a broken item (shipping damage), a **replacement should be emphasized over refund**
   - The customer wants the product - they just want an intact one
   - Offering refund equally with replacement may increase refund rates unnecessarily (contradicts policy goal to "minimize refund abuse")

2. **Tone and Empathy Could Be Stronger**
   - Opening apology is somewhat generic: "I'm sorry to hear..." 
   - Lacks acknowledgment that this is entirely the company's fault (shipping/packaging issue)
   - The phrase "if you are within our 14-day return window" sounds conditional and defensive when dealing with a defective product
   - Missing urgency and ownership of the problem

3. **Customer Experience**
   - Requires multiple steps from frustrated customer (photo, order number, date)
   - No immediate commitment to resolution
   - Feels bureaucratic rather than solution-focused

---

## Recommended Improvements

**Better Response Structure:**

```
Dear Valued Customer,

I sincerely apologize that your ceramic mug arrived broken. This is completely unacceptable, and I understand how frustrating this must be.

We'll send you a replacement immediately. To process this quickly, could you please provide:
- Your order number
- A photo of the damaged mug

If you'd prefer a refund instead, I can arrange that as well, provided your purchase was within the last 14 days.

I'm personally ensuring this gets resolved today. Thank you for your patience.

Best regards,
Customer Service Team
```

**Key Changes:**
- Stronger apology and ownership
- **Replacement prioritized** (aligns with customer satisfaction + minimizes refunds)
- More proactive and less conditional language
- Streamlined information request
- Greater sense of urgency

---

## Final Verdict

The original response **follows policy** but lacks the empathy and customer-centric approach needed for a clear-cut defective product case. It's functional but could damage customer loyalty due to its bureaucratic tone.

# **Your Challenge!!!**

A customer has submitted a complaint. Based on company policy and the situation, the model must select the correct resolution option from a list, and give a 1-line justification.

In [58]:
complaint = "I ordered a ceramic mug and received a broken one. Please fix this!"

policy = """
- Broken or damaged items must be reported within 7 days of delivery.
- If reported in time, the item is replaced or refunded.
- No action is taken for reports after 7 days unless escalated.
"""

metadata = """
    Reported Days After Delivery: 5
    Item Value: $12
    Customer Tier: Standard
    Past Complaints: 0
"""


Build a customer support agent that could help solve this issue

The agent should choose one of the following options:
- Refund Item
- Replace Item
- Offer Discount
- Apologize (no action)
- Report to supervisor

In [70]:
system_message = "You are a senior customer service representative with advanced reasoning capabilities. This includes analyzing customer complaints and providing detailed responses based on company policy."

prompt = f"""
A customer has made a complaint below:

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

More details about the customer case:
{metadata}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint, company policy, and customer metadata
- Professional tone and empathy

"""

message = client.messages.create(
    model="claude-sonnet-4-5-20250929", #claude-3-7-sonnet-latest",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    temperature=1,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
reasoning_message = message.content
display(Markdown(reasoning_message[1].text))

# Analysis of Customer Service Response

## Overall Assessment: **ADEQUATE BUT COULD BE IMPROVED**

---

## 1. Policy Compliance ✓

**Strengths:**
- Correctly references the 14-day return window
- Offers both replacement and refund options (appropriate for the situation)
- Requests verification documentation (photo and order number) which aligns with minimizing refund abuse

**Concerns:**
- The customer is clearly within policy (5 days vs. 14-day window)
- Item value is $12 (well below the $100 supervisor approval threshold)
- Customer has zero past complaints, indicating trustworthiness
- The response treats this as a "maybe" situation when it's clearly a legitimate claim

---

## 2. Professional Tone and Empathy

**Strengths:**
- ✓ Apologetic and acknowledges the issue
- ✓ Professional language and structure
- ✓ Shows willingness to resolve

**Weaknesses:**
- **Conditional language is problematic**: "*if* you are within our 14-day return window" sounds defensive and creates unnecessary doubt
- Lacks urgency despite the customer's direct request to "fix this!"
- Feels transactional rather than genuinely empathetic

---

## 3. Customer Experience Considerations

**Issues:**
- **Overly bureaucratic** for a low-value item ($12)
- Creates multiple friction points (send photo → wait → verify → then get resolution)
- Given the customer's clean record (0 past complaints) and the clear validity of the claim, this approach is unnecessarily cautious
- The phrase "verify your eligibility" may feel insulting to a legitimate customer

---

## Recommended Improved Response:

```
Dear Valued Customer,

I sincerely apologize that your ceramic mug arrived broken – that's certainly not the quality experience we want you to have.

I'd like to send you a replacement right away. To process this immediately, could you please provide:
- Your order number
- Your preferred shipping address (if different from original)

If you'd prefer a refund instead, I'm happy to arrange that as well.

For our records, if you could attach a quick photo of the damaged item when you reply, that would be helpful – but I won't hold up your replacement waiting for it.

Thank you for your patience, and again, my apologies for this inconvenience.

Best regards,
Customer Service Team
```

---

## Key Improvements Needed:

1. **Be more decisive** - Customer is clearly eligible, so act confidently
2. **Remove conditional language** - Don't create doubt with "if" statements
3. **Show urgency** - Offer immediate action rather than a multi-step verification process
4. **Balance policy with pragmatism** - For a $12 item with a clean customer record, err on the side of customer satisfaction
5. **Make the photo request optional** - Get it for records but don't gate the resolution on it

**Final Verdict:** The original response is professionally adequate but misses an opportunity to create a positive customer experience by being overly cautious in a clear-cut, low-risk situation.

## **Reflections**

1. What reasoning steps did the model take to select its resolution? Were they logical and aligned with company policy? (Hint: Have a look at the reasoning output)
- Select proper reasoning model
- Provide sufficient context in prompt, i.e. ensure adding complete and relevant info; add improvement suggestions from evaluation results to the chat prompt as being evaluated

2. If you were designing the agent, what would you change about its decision-making process?
- Add evaluation steps to ensure responses from the model being inline with the goal of the agent
- Pay intention to the evaluation results and adjust related prompts

3. What did you learn about prompt design and reasoning models from this exercise?
- The model for chat is different from the model in reasoning
- Evaluation of response from model is necessary to improve model's response
- How to perform iteration process based on the results of evaluation